In [1]:
%pip install jupyterlab pandas numpy matplotlib cartopy h5py folium geopandas xarray netcdf4 rasterio requests

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 10.5 MB/s  0:00:01
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'error'


  error: subprocess-exited-with-error
  
  × Preparing metadata (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [89 lines of output]
      configuration error: `project.license` must be valid exactly by one definition (2 matches found):
      
          - keys:
              'file': {type: string}
            required: ['file']
          - keys:
              'text': {type: string}
            required: ['text']
      
      DESCRIPTION:
          `Project license <https://peps.python.org/pep-0621/#license>`_.
      
      GIVEN VALUE:
          "BSD-3-Clause"
      
      OFFENDING RULE: 'oneOf'
      
      DEFINITION:
          {
              "oneOf": [
                  {
                      "properties": {
                          "file": {
                              "type": "string",
                              "$description": [
                                  "Relative path to the file (UTF-8) which contains the license for the",
                    

Note: you may need to restart the kernel to use updated packages.


# WarnWetter App Data Exploration with DWD API

This notebook aims to explore the data available from the Deutscher Wetterdienst (DWD) and Bright Sky APIs to reproduce visualizations similar to those found in the WarnWetter app. We will focus on radar data and potentially weather warnings.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import h5py
import folium
import geopandas as gpd
import xarray as xr
import netCDF4
import rasterio
import requests

## 1. Data Acquisition from DWD Open Data Server
The Deutscher Wetterdienst (DWD) provides a wealth of meteorological data through its Open Data Server. For radar data, we are interested in the `weather/radar/composite/dmax/` directory, which contains composite radar images in HDF5 format.
We can construct the URL to download these files. The file names typically follow a pattern like `composite_dmax_YYYYMMDD_HHMM-hd5`, where YYYYMMDD is the date and HHMM is the time.

In [ ]:
# Example: Download a sample radar data file
base_url = "https://opendata.dwd.de/weather/radar/composite/dmax/"
file_name = "composite_dmax_20250802_2000-hd5"  # Replace with a recent file from the DWD server
download_url = base_url + file_name
output_path = file_name

print(f"Downloading {download_url} to {output_path}...")
try:
    response = requests.get(download_url, stream=True)
    response.raise_for_status()
    with open(output_path, "wb") as f:
        for chunk in response.iter_content(chunk_size=8192):
            f.write(chunk)
    print("Download complete.")
except requests.exceptions.RequestException as e:
    print(f"Error downloading file: {e}")

# Verify the downloaded file (optional)
# import os
# if os.path.exists(output_path):
#     print(f"File size: {os.path.getsize(output_path)} bytes")
# else:
#     print("File not found.")

## 2. Data Exploration and Initial Visualization
After downloading the HDF5 radar data, we can use the `h5py` library to inspect its contents. The radar data typically contains precipitation intensity information that can be visualized on a map.
We will use `matplotlib` and `cartopy` to create a basic static visualization of the radar data. This will help us understand the data structure and its geographical representation.

In [ ]:
# Load the HDF5 file
try:
    with h5py.File(output_path, "r") as f:
        # List all groups and datasets
        print("Keys: %s" % f.keys())
        data = f["/dwd/radar/composite/dmax/data"][()]
        attrs = f["/dwd/radar/composite/dmax/data"].attrs
        lon = f["/dwd/radar/composite/dmax/lon"][()]
        lat = f["/dwd/radar/composite/dmax/lat"][()]
        
        print("Data shape:", data.shape)
        print("Longitude shape:", lon.shape)
        print("Latitude shape:", lat.shape)
        print("Attributes:", dict(attrs))

        # Basic visualization
        fig = plt.figure(figsize=(10, 10))
        ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())
        ax.set_extent([lon.min(), lon.max(), lat.min(), lat.max()], crs=ccrs.PlateCarree())
        ax.coastlines(resolution="10m")
        ax.gridlines()
        
        # Plot the radar data
        # Assuming data is in dBZ, convert to mm/h if necessary for better visualization
        # This conversion might vary based on the specific DWD product
        # For dmax, it\"s typically already in mm/h or a related unit, check documentation
        # For now, just plot directly
        im = ax.imshow(data, origin="lower", extent=[lon.min(), lon.max(), lat.min(), lat.max()],
                       cmap="viridis", transform=ccrs.PlateCarree())
        plt.colorbar(im, ax=ax, orientation="vertical", label="Precipitation Intensity")
        plt.title("DWD Radar Composite (dmax)")
        plt.show()
except Exception as e:
    print(f"Error processing HDF5 file: {e}")

## 3. Refined Visualization and Data Interpretation
The `dmax` product from DWD radar data represents the maximum reflectivity in a column, often used to infer precipitation intensity. The units and scaling can sometimes be tricky. It\"s important to consult DWD documentation for precise interpretation. For visualization, we can adjust color mapping and add geographical features for better context.
Let\"s refine the `matplotlib` visualization and also consider an interactive map using `folium`.

In [ ]:
# Refined Matplotlib Visualization
try:
    with h5py.File(output_path, "r") as f:
        data = f["/dwd/radar/composite/dmax/data"][()]
        lon = f["/dwd/radar/composite/dmax/lon"][()]
        lat = f["/dwd/radar/composite/dmax/lat"][()]

        fig = plt.figure(figsize=(12, 12))
        ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())
        ax.set_extent([lon.min(), lon.max(), lat.min(), lat.max()], crs=ccrs.PlateCarree())
        ax.coastlines(resolution="10m", color="black", linewidth=1)
        ax.add_feature(cartopy.feature.BORDERS, linestyle=":", edgecolor="gray")
        ax.add_feature(cartopy.feature.LAKES, alpha=0.5)
        ax.add_feature(cartopy.feature.RIVERS)
        
        # Choose a colormap suitable for precipitation (e.g., "viridis", "jet", "Blues")
        # You might need to adjust vmin and vmax based on the data range
        im = ax.imshow(data, origin="lower", extent=[lon.min(), lon.max(), lat.min(), lat.max()],
                       cmap="jet", vmin=0, vmax=50, transform=ccrs.PlateCarree())
        plt.colorbar(im, ax=ax, orientation="vertical", label="Precipitation Intensity (dBZ or mm/h)")
        plt.title("DWD Radar Composite (dmax) - Refined Visualization")
        plt.show()
except Exception as e:
    print(f"Error processing HDF5 file for refined visualization: {e}")

### Interactive Map with Folium (Conceptual)
For interactive visualizations, `folium` can be used. However, directly overlaying raster data from HDF5 onto `folium` requires converting the data into a format `folium` can understand, such as a GeoTIFF or a series of image tiles. This is more complex and might be beyond the scope of a quick exploration without a dedicated tiling server.
A simpler approach for `folium` would be to plot geographical points or polygons, which is not directly applicable to raw radar raster data without significant preprocessing. Therefore, we will focus on `matplotlib` for raster visualization.

## 4. Animation of Weather Radar Data
To create an animation similar to the WarnWetter app, we need to download a sequence of radar data files over a period of time. Then, we can use `matplotlib.animation` to create a GIF or a video of the radar data changing over time.
For this example, we will simulate downloading multiple files by assuming a list of file names. In a real-world scenario, you would dynamically generate these filenames based on the desired time range and available data on the DWD server.

In [ ]:
from matplotlib.animation import FuncAnimation, PillowWriter
import os
import datetime

# --- Configuration for animation ---
# Define a time range for the animation (e.g., last 2 hours, every 5 minutes)
# In a real scenario, you would fetch available files from DWD server listing
end_time = datetime.datetime.now()
start_time = end_time - datetime.timedelta(hours=2) # Last 2 hours
interval_minutes = 5

# Generate a list of hypothetical file names (replace with actual logic to fetch available files)
# This is a placeholder. You would need to parse the DWD directory listing for actual files.
# For demonstration, let\"s assume we have files for every 5 minutes in the last 2 hours.
file_timestamps = []
current_time = start_time
while current_time <= end_time:
    file_timestamps.append(current_time.strftime("%Y%m%d_%H%M"))
    current_time += datetime.timedelta(minutes=interval_minutes)

radar_files = [f"composite_dmax_{ts}-hd5" for ts in file_timestamps]

# Create a directory to store downloaded radar files for animation
animation_data_dir = "./radar_animation_data"
os.makedirs(animation_data_dir, exist_ok=True)

# Download all files for animation (this will take time and bandwidth)
downloaded_files = []
for fname in radar_files:
    download_url = base_url + fname
    output_path_anim = os.path.join(animation_data_dir, fname)
    if not os.path.exists(output_path_anim):
        print(f"Downloading {download_url} to {output_path_anim}...")
        try:
            response = requests.get(download_url, stream=True)
            response.raise_for_status()
            with open(output_path_anim, "wb") as f:
                for chunk in response.iter_content(chunk_size=8192):
                    f.write(chunk)
            print("Download complete.")
            downloaded_files.append(output_path_anim)
        except requests.exceptions.RequestException as e:
            print(f"Error downloading {fname}: {e}")
    else:
        print(f"File {fname} already exists, skipping download.")
        downloaded_files.append(output_path_anim)

# --- Animation Code ---
if downloaded_files:
    fig = plt.figure(figsize=(12, 12))
    ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())
    ax.set_extent([lon.min(), lon.max(), lat.min(), lat.max()], crs=ccrs.PlateCarree())
    ax.coastlines(resolution="10m", color="black", linewidth=1)
    ax.add_feature(cartopy.feature.BORDERS, linestyle=":", edgecolor="gray")
    ax.add_feature(cartopy.feature.LAKES, alpha=0.5)
    ax.add_feature(cartopy.feature.RIVERS)

    # Initialize the plot with the first frame
    with h5py.File(downloaded_files[0], "r") as f:
        initial_data = f["/dwd/radar/composite/dmax/data"][()]
    im = ax.imshow(initial_data, origin="lower", extent=[lon.min(), lon.max(), lat.min(), lat.max()],
                   cmap="jet", vmin=0, vmax=50, transform=ccrs.PlateCarree())
    title = ax.set_title(f"DWD Radar Composite (dmax) - {file_timestamps[0]}")

    def update(frame):
        try:
            with h5py.File(downloaded_files[frame], "r") as f:
                data = f["/dwd/radar/composite/dmax/data"][()]
            im.set_array(data)
            title.set_text(f"DWD Radar Composite (dmax) - {file_timestamps[frame]}")
        except Exception as e:
            print(f"Error loading frame {frame}: {e}")
        return [im, title]

    ani = FuncAnimation(fig, update, frames=len(downloaded_files), interval=200, blit=True)

    # Save the animation as a GIF
    gif_path = "radar_animation.gif"
    print(f"Saving animation to {gif_path}...")
    ani.save(gif_path, writer=PillowWriter(fps=5))
    print("Animation saved.")
    plt.close(fig) # Close the plot to prevent it from displaying after saving
else:
    print("No radar files downloaded for animation.")